In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
import os
import torch
from torch import nn
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import seaborn as sns

# import multiple cities worth of data
# data_5 = pd.read_csv('data/Austin_Final_2022-06-18.csv')
# data_6 = pd.read_csv('data/Albuquerque_Final_2022-06-18.csv')
# data_3 = pd.read_csv('data/StLouis_Final_2022-06-18.csv')
# data_4 = pd.read_csv('data/WashingtonDC_Final_2022-06-18.csv')
data_1 = pd.read_csv('data/NewYork_Final_2022-06-18.csv')
data_2 = pd.read_csv('data/NewYork_Final_2022-06-18_(2).csv')

# list of dataframes
data_list = [data_1, data_2]


In [2]:
print(torch.__version__)

2.0.0+cu117


: 

In [2]:
# loop through the dataframes in data_list
for i in range(1, len(data_list) + 1):
    # get the dataframe
    data = globals()['data_' + str(i)]
    
    # one hot tree types
    tree_types  = pd.get_dummies(data['common_name'])

    # drop the common_name column
    data = data.drop(columns=['common_name'])

    # one hot condition column
    data['condition'] = data['condition'].replace({'excellent': 0, 'good': 1, 'fair': 2, 'poor': 3, 'dead/dying': 4, 'dead': 4})
    
    # create new dataframe with latitude and longitude, tree_types, native, and condition
    all_data = pd.concat([data[['longitude_coordinate', 'latitude_coordinate', 
                                'condition']],
                            tree_types],
                            axis=1)

    # drop rows with NaN values
    all_data = all_data.dropna()

    # save the dataframe
    globals()['all_data_' + str(i)] = all_data

# concatenate all the dataframes
all_data = pd.concat([all_data_1, all_data_2], axis=0)


# # one hot tree types
# tree_types  = pd.get_dummies(data['common_name'])

# # convert condition to numerical
# condition = data['condition'].replace({'excellent': 0, 'good': 1, 'fair': 2, 'poor': 3, 'dead/dying': 4, 'dead': 4})

# # one hot native column
# native = pd.get_dummies(data['native'])

# # create new dataframe with latitude and longitude, tree_types, native, and condition
# all_data = pd.concat([data[['longitude_coordinate', 'latitude_coordinate']], 
#                       tree_types, 
#                       native, 
#                       condition], 
#                       axis=1)


all_data.head()



,longitude_coordinate,latitude_coordinate,condition,American beech,American elm,American hophornbeam,American hornbeam,American larch,American linden,Amur cork tree,...,Virginia pine,Weeping willow,White ash,White oak,White pine,Willow oak,Winged silverbell two,False cypress,Mimosa,Pitch pine
0,-73.844215,40.723092,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,-73.818679,40.794111,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,-73.936608,40.717581,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,-73.934456,40.713537,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,-73.975979,40.666778,1.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [3]:
# split data back into x and y
x_data = all_data.drop(['condition'], axis=1)
y_data = all_data['condition']

# print unique values in y_data
print(y_data.unique())

# remove NaN values
x_data = x_data.dropna()

# print if there are any NaN values in x_data
print(x_data.isnull().values.any())

[2. 1. 3.]
False


In [4]:
# # standardize x_data
# scaler = StandardScaler()
# x_data = scaler.fit_transform(x_data)

In [5]:
# split into train and test
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=25)

# drop NaN values
train_data = train_data.dropna()
test_data = test_data.dropna()

In [6]:
# split into x and y
x_train = train_data.drop(['condition'], axis=1)
y_train = train_data['condition']
x_test = test_data.drop(['condition'], axis=1)
y_test = test_data['condition']

In [7]:
# # compute correlation matrix
# corr_df = x_train
# cor = corr_df.corr(method='pearson') 
# # print(cor)

# # Plot
# fig_corr, ax_corr =plt.subplots()
# plt.title("Correlation Plot")
# sns.heatmap(cor, cmap=plt.cm.RdYlBu, square=True, ax=ax_corr)
# plt.show()


KeyboardInterrupt: 

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# train logistic regression model on training data
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

# predict on test data
y_pred = logistic_regression.predict(x_test)

# compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)



0.8191193911098434


/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# optimize logistic regression model
from sklearn.model_selection import GridSearchCV

# initialize logistic regression model parameters
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)   

# create empty accuracy list
accuracy = []

# loop through parameters
for p in penalty:
    for c in C:
        # initialize logistic regression model
        logistic_regression = LogisticRegression(penalty=p, C=c, solver='liblinear')
        
        # fit model
        logistic_regression.fit(x_train, y_train)
        
        # predict on test data
        y_pred = logistic_regression.predict(x_test)
        
        # compute accuracy
        accuracy.append(accuracy_score(y_test, y_pred))

# # find max accuracy
# max_accuracy = max(accuracy)
# print(max_accuracy)

# # find index of max accuracy
# max_accuracy_index = accuracy.index(max_accuracy)
# print(max_accuracy_index)

# find parameters that correspond to max accuracy
penalty = penalty[max_accuracy_index // len(C)]
C = C[max_accuracy_index % len(C)]

# plot accuracy
plt.plot(accuracy)
plt.show()






/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alexisfrankson/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/alexisfrankson/opt/anaconda3

NameError: name 'max_accuracy_index' is not defined

In [ ]:
# return optimal parameters
print(penalty)
print(C)

# initialize logistic regression model with optimal parameters
logistic_regression = LogisticRegression(penalty=penalty, C=C, solver='liblinear')

# fit model
logistic_regression.fit(x_train, y_train)

# predict on test data
y_pred = logistic_regression.predict(x_test)

# compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

l1
5214.00828799969
0.5203389830508475


In [ ]:
x_train = x_train.ToTensor()
y_train = y_train.ToTensor()
x_test = x_test.ToTensor()
y_test = y_test.ToTensor()

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits